In [1]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "anywidget>=0.9.0",
#     "jupyter-scatter-scsketch>=0.21.0",
#     "llvmlite>=0.44.0",
#     "numpy>=1.26.0",
#     "pandas>=2.0.0",
#     "scanpy>=1.9.0",
#     "scipy>=1.11.0",
#     "ipywidgets>=8.0.0",
#     "matplotlib>=3.7.0",
#     "requests>=2.31.0",
#     "watchfiles>=0.20.0",
# ]
#
# [tool.uv.sources]
# scsketch = { path = ".", editable = true }
# ///

# scSketch

> **Quick Start**: `uvx scsketch demo` (no cloning required!)  
> Or from this repo: `uvx juv run demo.ipynb`

scSketch provides a custom UI for [Jupyter-Scatter](https://jupyter-scatter.dev) that implements [Directional Analysis from Colubri et al's Sciviewer](https://doi.org/10.1093/bioinformatics/btab689). Sciviewer's directional analysis helps you interpret patterns in embedding visualizations by identifying genes varying locally along any user-specified direction.

For this demo, we're using the PBMC 3k (processed) dataset, since it is small (~2,600 cells), publicly available, and explicitly contains the UMAP coordinates and cluster metadata.

## Load Data

Load your single-cell data using scanpy and prepare it for visualization.

In [2]:
# Dataset download (shows a widget progress bar while downloading)

import os.path
import urllib.request

import ipywidgets as ipyw
from IPython.display import display

# Pick a dataset:
# data_file = "pbmc3k.h5ad"
data_file = "nygc_pbmc_161k_lite.h5ad"
# data_file = "1M_20260121.h5ad"
data_url = None  # set to a URL to download if the file is missing
# data_file = "pbmc3k.h5ad"
# data_url = "https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad"

status = ipyw.HTML("")
bar = ipyw.IntProgress(value=0, min=0, max=100, description="Download", layout=ipyw.Layout(width="100%"))
ui = ipyw.VBox([status])
display(ui)

def _format_bytes(n: int) -> str:
    n = float(n)
    for unit in ["B", "KB", "MB", "GB", "TB"]:
        if n < 1024 or unit == "TB":
            return f"{int(n)} {unit}" if unit == "B" else f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

if os.path.exists(data_file):
    size = os.path.getsize(data_file)
    status.value = f"<b>Dataset:</b> Found <code>{data_file}</code> ({_format_bytes(size)})."
else:
    if not data_url:
        status.value = f"<b>Dataset:</b> Missing <code>{data_file}</code>. Set <code>data_url</code> to download."
    else:
        ui.children = [status, bar]
        status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code>…"

        def _hook(blocknum: int, blocksize: int, totalsize: int):
            downloaded = blocknum * blocksize
            if totalsize and totalsize > 0:
                pct = int(min(100, downloaded * 100 / totalsize))
                bar.value = pct
                status.value = (
                    f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {pct}% "
                    f"({_format_bytes(min(downloaded, totalsize))} / {_format_bytes(totalsize)})"
                )
            else:
                status.value = f"<b>Dataset:</b> Downloading <code>{data_file}</code> — {_format_bytes(downloaded)}"

        urllib.request.urlretrieve(data_url, data_file, reporthook=_hook)
        bar.value = 100
        size = os.path.getsize(data_file)
        status.value = f"<b>Dataset:</b> Downloaded <code>{data_file}</code> ({_format_bytes(size)})."


In [3]:
import scanpy as sc

adata = sc.read(data_file)
adata

AnnData object with n_obs × n_vars = 161764 × 20525
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor_id', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT', 'cell_type_ontology_term_id', 'sex_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'assay_ontology_term_id', 'is_primary_data', 'tissue_ontology_term_id', 'suspension_type', 'tissue_type', 'cell_type', 'assay', 'disease', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'observation_joinid'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'feature_length', 'feature_type'
    uns: 'celltype.l3_colors', 'citation', 'default_embedding', 'neighbors', 'organism', 'organism_ontology_term_id', 'schema_reference', 'schema_version', 'title'
    obsm: 'X_apca', 'X_aumap', 'X_pca', 'X_spca', 'X_umap', 'X_wnn.umap'
    va

## Launch scSketch

Create and display the scSketch widget.

In [4]:
from scsketch import ScSketch

sketch = ScSketch(
    adata=adata,
    metadata_cols=["cell_type"],   # Metadata columns that can be used to color the cells
    color_by_default="cell_type",  # A metadata column to use by default for coloring
    height=720,
    background_color="#111111",
)
sketch.show()

In [5]:
!MPLCONFIGDIR=/tmp/mplconfig XDG_CACHE_HOME=/tmp/xdgcache .venv/bin/python benchmarks/perf_retest.py --out benchmarks/perf_results_numba.csv

/Users/askartemirbek/anywidgets_practice/scsketch/.venv/lib/python3.12/site-packages/anndata/_io/h5ad.py:266: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  return AnnData(**{
/Users/askartemirbek/anywidgets_practice/scsketch/.venv/lib/python3.12/site-packages/anndata/_io/h5ad.py:266: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  return AnnData(**{
Wrote 9 rows to benchmarks/perf_results_numba.csv


In [6]:
sketch.get_genes("Selection 1") # Get the list of selected genes as a dataframe for further analysis

""


In [10]:
adata.var.head()

,gene_symbols,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length,feature_type
ENSG00000243485,MIR1302-2HG,False,MIR1302-2HG,NCBITaxon:9606,gene,517,lncRNA
ENSG00000237613,FAM138A,False,FAM138A,NCBITaxon:9606,gene,1015,lncRNA
ENSG00000186092,OR4F5,False,OR4F5,NCBITaxon:9606,gene,2618,protein_coding
ENSG00000239945,AL627309.3,False,ENSG00000239945,NCBITaxon:9606,gene,1319,lncRNA
ENSG00000239906,AL627309.2,False,ENSG00000239906,NCBITaxon:9606,gene,323,lncRNA


In [6]:
#Since scSketch by default pulls the Ensembl IDs for the genes for the ~1.2M cell dataset, because Reactome expects UniProt IDs, here is the code to enable scSketch to present UniProt IDs in the results so Reactome pathway lookups can work.  

adata_view = adata.copy()

adata_view.var["ensembl_id"] = adata_view.var_names
adata_view.var_names = adata_view.var["gene_symbols"].astype(str)
adata_view.var_names_make_unique()  # important if symbols repeat

from scsketch import ScSketch
sketch = ScSketch(adata=adata_view, metadata_cols=["cell_type"], color_by_default="cell_type", verbosity="debug")
sketch.show()

KeyError: 'gene_symbols'

In [ ]:
from scsketch import ScSketch
sketch = ScSketch(adata=adata_view, metadata_cols=["cell_type"], color_by_default="cell_type", verbosity="debug")
sketch.show()

In [20]:
import scipy.sparse as sp
sel = sketch.active_selection.points
X_sel = adata.X[sel, :]
print(type(X_sel), getattr(X_sel, "format", None), X_sel.shape, getattr(X_sel, "nnz", None))

<class 'scipy.sparse._csr.csr_matrix'> csr (376364, 32357) 577500417


In [18]:
sketch.get_genes("Selection 1") # Get the list of selected genes as a dataframe for further analysis

,gene,correlation,p-value
7785,RPS13,0.670550,0.0
3760,RPS3A,0.654269,0.0
2586,RPL32,0.629651,0.0
220,RPL11,0.611933,0.0
67,RPL22,0.611828,0.0
...,...,...,...
3964,GZMA,-0.655907,0.0
10213,GZMH,-0.686699,0.0
13100,CST7,-0.724245,0.0
12169,CCL5,-0.733406,0.0


In [21]:
adata

AnnData object with n_obs × n_vars = 1206761 × 32357
    obs: 'original_barcodes', 'cell_name', 'batch_id', 'pool_id', 'chip_id', 'well_id', 'n_genes', 'n_reads', 'n_umis', 'total_counts_mito', 'pct_counts_mito', 'doublet_score', 'predicted_AIFI_L1', 'AIFI_L1_score', 'AIFI_L1', 'predicted_AIFI_L2', 'AIFI_L2_score', 'AIFI_L2', 'predicted_AIFI_L3', 'AIFI_L3_score', 'AIFI_L3', 'sample.sampleKitGuid', 'cohort.cohortGuid', 'subject.subjectGuid', 'subject.cmv', 'subject.bmi', 'subject.race', 'subject.ethnicity', 'subject.birthYear', 'subject.ageAtFirstDraw', 'sample.visitName', 'sample.drawYear', 'sample.subjectAgeAtDraw', 'specimen.specimenGuid', 'pipeline.fileGuid', 'subject.ageGroup', 'vaccine_year', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'tissue_type', 'suspension_type', 'assay_ontology_term_id', 'sex_ontology_term_id', 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'cell_type_ontology_term_id', 'is_primary_data', 'cell_type'

In [10]:
import numpy as np, os
os.getcwd()

'/Users/askartemirbek/anywidgets_practice/scsketch'

In [12]:
import numpy as np
sel = np.asarray(sketch.active_selection.points, dtype=int)

np.savez(
  "benchmarks/selection_1M_20260121_sel1.npz",
  dataset="1.2m",
  name="sel1",
  indices=sel,
)

In [13]:
import os
os.path.exists("benchmarks/selection_1M_20260121_sel1.npz")

True

In [14]:
!uv run python benchmarks/perf_retest.py --datasets 1.2m --selection-npz benchmarks/selection_1M_20260121_sel1.npz --no-synthetic --out benchmarks/perf_results.csv

In [16]:
import numpy as np

z = np.load("benchmarks/selection_1M_20260121_sel1.npz")
print(z.files)
print("dataset:", z["dataset"])
print("name:", z["name"])
print("indices shape:", z["indices"].shape, "dtype:", z["indices"].dtype)
print("first10:", z["indices"][:10])

['dataset', 'name', 'indices']
dataset: 1.2m
name: sel1
indices shape: (270146,) dtype: int64
first10: [ 229962 1076965  808303 1115199  353344  698529  699274  218004  560150
  717967]


In [23]:
!uv run python benchmarks/perf_retest.py --datasets 1.2m --selection-npz benchmarks/selection_1M_20260121_sel1.npz --no-synthetic --out benchmarks/perf_results_exported.csv

python3.13(8434) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Wrote 1 rows to benchmarks/perf_results_exported.csv


In [24]:
!ls -lt benchmarks/perf_results*.csv
!head -n 2 benchmarks/perf_results_exported.csv

-rw-r--r--@ 1 askartemirbek  staff   373 Feb  2 11:13 benchmarks/perf_results_exported.csv
-rw-r--r--@ 1 askartemirbek  staff  1643 Feb  1 22:12 benchmarks/perf_results.csv


python3.13(9959) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python3.13(9963) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


dataset,file,embedding,selection_source,selection_name,n_obs,n_vars,X_type,X_nnz_total,n_selected,X_sel_type,X_sel_nnz,directional_corr_p_s,directional_lord_s,de_global_stats_s,de_compute_s
1.2m,1M_20260121.h5ad,X_umap,exported,sel1,1206761,32357,csr_matrix,2056839573,270146,csr_matrix,419545342,6.144334707874805,25.73643287504092,249.67604716634378,16.903598040807992


In [20]:
import numpy as np

z = np.load("benchmarks/selection_1M_20260121_sel1.npz")
print(z.files)
print("dataset:", z["dataset"])
print("name:", z["name"])
print("indices shape:", z["indices"].shape, "dtype:", z["indices"].dtype)
print("first10:", z["indices"][:10])

['dataset', 'name', 'indices']
dataset: 1.2m
name: sel1
indices shape: (270146,) dtype: int64
first10: [ 229962 1076965  808303 1115199  353344  698529  699274  218004  560150
  717967]


In [21]:
!ls -lt benchmarks | head

total 4272
-rw-r--r--@ 1 askartemirbek  staff  2161954 Feb  2 10:21 selection_1M_20260121_sel1.npz
-rw-r--r--@ 1 askartemirbek  staff     2065 Feb  2 10:02 README.md
-rw-r--r--@ 1 askartemirbek  staff    14114 Feb  2 10:02 perf_retest.py
-rw-r--r--@ 1 askartemirbek  staff     1643 Feb  1 22:12 perf_results.csv
drwxr-xr-x@ 3 askartemirbek  staff       96 Feb  1 21:41 __pycache__


python3.13(8196) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [22]:
!head -n 2 benchmarks/perf_results_exported.csv

head: benchmarks/perf_results_exported.csv: No such file or directory


python3.13(8277) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [17]:
len(sketch.active_selection.points)

376364

## How to Use scSketch

1. **Select points**: Use the rectangle or lasso tool to select cells in the embedding
2. **Add selection**: Click the `+` button to save your selection
3. **Run analysis**: Click `Compute Directional Search` to identify genes varying along the selected direction
4. **Explore results**: Click on genes to see their Reactome pathways, and click pathways to view diagrams
5. **Color by genes**: Use the dropdown to color the embedding by specific genes or metadata
6. **Get list genes**: Use the get_genes() function in the scSketch object to retrieve the list of genes in a selected direction.

The directional analysis shows genes with their Pearson Correlation Coefficient (R) and p-value (p), representing which genes are most upregulated or downregulated along the selected direction.